In [27]:
import pandas as pd

In [28]:
df = pd.read_excel("./kospi.xlsx", usecols=[0,1,2,3,4], index_col='Date')
df.head(1)

,Close,Open,High,Low
Date,,,,
2000-01-04,1059.04,1028.33,1066.18,1016.59


In [29]:
# 변동 파생변수
df['change'] = df['High'] - df['Low']
df.head(1)

,Close,Open,High,Low,change
Date,,,,,
2000-01-04,1059.04,1028.33,1066.18,1016.59,49.59


In [30]:
# 전일변동
df['전일변동'] = df['change'].shift()
df.head()

,Close,Open,High,Low,change,전일변동
Date,,,,,,
2000-01-04,1059.04,1028.33,1066.18,1016.59,49.59,NaN
2000-01-05,986.31,1006.87,1026.52,984.05,42.47,49.59
2000-01-06,960.79,1013.95,1014.90,953.50,61.40,42.47
2000-01-07,948.65,949.17,970.16,930.84,39.32,61.40
2000-01-10,987.24,979.67,994.94,965.02,29.92,39.32


In [31]:
# 목표가 = 시가 + 전일변동 * 0.5
df['목표가'] = df['Open'] + df['전일변동'] * 0.5

In [35]:
df

,Close,Open,High,Low,change,전일변동,목표가
Date,,,,,,,
2000-01-04,1059.04,1028.33,1066.18,1016.59,49.59,NaN,NaN
2000-01-05,986.31,1006.87,1026.52,984.05,42.47,49.59,1031.665
2000-01-06,960.79,1013.95,1014.90,953.50,61.40,42.47,1035.185
2000-01-07,948.65,949.17,970.16,930.84,39.32,61.40,979.870
2000-01-10,987.24,979.67,994.94,965.02,29.92,39.32,999.330
...,...,...,...,...,...,...,...
2021-08-23,3090.21,3086.81,3111.81,3064.65,47.16,73.06,3123.340
2021-08-24,3138.30,3119.70,3142.58,3115.23,27.35,47.16,3143.280
2021-08-25,3146.81,3148.13,3161.00,3124.25,36.75,27.35,3161.805


In [42]:
df.loc['2000-01-05','목표가']

1031.665

1. 목표가보다 고가가 높으면 목표가로 매수
2. 수익율 -> 종가 - 목표가 / 목표가 + 1
3. 그 외의 수익율은 1

In [45]:
acc_rtn =1
for i in df.index:
    if df.loc[i,'목표가'] < df.loc[i,'High']:
        df.loc[i,'수익율'] = (df.loc[i,'Close'] - df.loc[i,'목표가'])/ df.loc[i,'목표가'] + 1
        
    else:
        df.loc[i,'수익율'] = 1
    acc_rtn *= df.loc[i,'수익율']
    df.loc[i, '누적수익율'] = acc_rtn

    


In [46]:
df

,Close,Open,High,Low,change,전일변동,목표가,수익율,누적수익율
Date,,,,,,,,,
2000-01-04,1059.04,1028.33,1066.18,1016.59,49.59,NaN,NaN,1.000000,1.000000
2000-01-05,986.31,1006.87,1026.52,984.05,42.47,49.59,1031.665,1.000000,1.000000
2000-01-06,960.79,1013.95,1014.90,953.50,61.40,42.47,1035.185,1.000000,1.000000
2000-01-07,948.65,949.17,970.16,930.84,39.32,61.40,979.870,1.000000,1.000000
2000-01-10,987.24,979.67,994.94,965.02,29.92,39.32,999.330,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...
2021-08-23,3090.21,3086.81,3111.81,3064.65,47.16,73.06,3123.340,1.000000,5.237280
2021-08-24,3138.30,3119.70,3142.58,3115.23,27.35,47.16,3143.280,1.000000,5.237280
2021-08-25,3146.81,3148.13,3161.00,3124.25,36.75,27.35,3161.805,1.000000,5.237280


In [47]:
import numpy as np

In [48]:
df['수익율2'] = np.where(df['High'] >= df['목표가'],
                    (df['Close']-df['목표가'])/ df['목표가'] + 1,
                    1)

In [50]:
(df['수익율'] == df['수익율2']).value_counts()

True    5349
dtype: int64

In [51]:
# 누적수익율
# 누적곱 -> cumprod()
df['누적수익율2'] = df['수익율2'].cumprod()

In [52]:
(df['누적수익율'] == df['누적수익율2']).value_counts()

True    5349
dtype: int64

In [53]:
#CAGR -> 연복리수익율
## 누적수익율 ** (1/(투자기간/365)) - 1
# 시계열데이터를 이용하여 기간
delta = df.index[-1] - df.index[0]
delta

Timedelta('7906 days 00:00:00')

In [54]:
delta.days

7906

In [55]:
year = delta.days/365
CAGR = df['누적수익율'].iloc[-1] ** (1/year) - 1
print('CAGR: ', CAGR * 100)

CAGR:  7.941722848934241
